# For different parameterizations of the two-spike Gaussian, determine the empirical probability that the estimate zetaHat(0) exceeds 1/2 zeta*

In [ ]:
from syntheticExperimentWrappers import drawAndEstimate_GaussianTwoSpike
import numpy as np
import matplotlib.pyplot as plt
from time import time
from multiprocessing import Pool, freeze_support
import pickle
import scipy
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os

In [ ]:
NUM_CORES = 40

In [ ]:
folderName = "HeatMapZetaHats"
if not os.path.exists(folderName):
    os.makedirs(folderName)

In [ ]:
def drawAndEstimate_GaussianTwoSpike_wrapper(threshold, n, zeta, mu2, discretization, sigma, tolerance, estimator, alpha):
    zetaHat = drawAndEstimate_GaussianTwoSpike(threshold, n, zeta, mu2, discretization, sigma, tolerance, estimator, alpha)
    return zetaHat, mu2, zeta

In [ ]:
maxJobs = NUM_CORES
n = 10000
discretization = 1000
tolerance = 0.001
sigma = 1
alpha = 0.05
numReplicates = 10
mu2s = np.linspace(0.1, 2, 30)
zetas = np.linspace(0.01, 0.5, 10)
threshold = 0

# Batching the jobs so we have intermediate saving. This is... not the BEST technique
while True:
    jobs = []
    # Get the job list
    for mu2 in mu2s:
        for zeta in zetas:
            try:  # So we can re-run this code if it stops halfway, without recomputing anything.
                zetaHats = pickle.load(open("./HeatMapZetaHats/zetaHats_"+str(n)+str(mu2)+str(zeta)+".p", 'rb'))
            except:
                if len(jobs) + numReplicates <= maxJobs:
                    jobs = jobs + [(threshold, n, zeta, mu2, discretization, sigma, tolerance, "KS", alpha) for _ in range(numReplicates)]

    if len(jobs) == 0:
        break   # No more jobs
        
    print(len(jobs), "jobs will be processed across", NUM_CORES, "cores")
    with Pool(NUM_CORES) as p:
        results = p.starmap(drawAndEstimate_GaussianTwoSpike_wrapper, jobs)
        
    for mu2 in mu2s:
        for zeta in zetas:
            zetaHats = [zetaHat for (zetaHat, m, z) in results if m==mu2 and z==zeta]
            if len(zetaHats) > 0:  # If we did this job in this round
                pickle.dump(zetaHats, open("./HeatMapZetaHats/zetaHats_"+str(n)+str(mu2)+str(zeta)+".p", 'wb'))

In [ ]:
FRAC=0.5

# Set up an array to hold the saved data
fracZetaHatsGtrHalf = np.zeros((len(mu2s), len(zetas)))

for i, gamma in enumerate(mu2s):
    for j, zeta in enumerate(zetas):
        zetaHats = pickle.load(open("./HeatMapZetaHats/zetaHats_"+str(n)+str(gamma)+str(zeta)+".p", 'rb'))
        fracZetaHatsGtrHalf[i, j] = sum([z > zeta*FRAC for z in zetaHats])/len(zetaHats)
        
   

In [ ]:
plt.rcParams["image.cmap"] = 'Blues'
plt.rcParams["font.size"] = 16
plt.rcParams["figure.figsize"] = [10,15]
ax = plt.subplot(111)
plt.tight_layout(pad=2)
im = ax.imshow(np.flipud(np.transpose(fracZetaHatsGtrHalf)), extent=(min(mu2s), max(mu2s), min(zetas), max(zetas)), vmin=0, vmax=1)
plt.xlabel(r'Effect size of alternate hypotheses, $\gamma_*$')
plt.ylabel(r'Fraction of non-null'+str("\n")+'hypotheses $\zeta_*$')

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size='2%', pad=-2.3)
plt.colorbar(im, cax=cax)

ax.set_aspect(1.8)
plt.tight_layout()
plt.savefig("HeatmapHalf.png")
plt.savefig("HeatmapHalf.eps")
plt.show()